# Resampling images

This notebook emphasizes that images that are sampled differently can be in the same physical
coordinate system, as long as their metadata are correctly set, and interpreted correctly.


First import the packages we will use:

In [ ]:
import numpy as np
import SimpleITK as sitk

from skimage.transform import rescale, resize, downscale_local_mean

Next we'll define a two functions that resample images in two different ways.

The first uses SimpleITK, the second uses scikit-image

Read through the comments for both functions, and get an idea for what both are doing.
It is not critical to understand the details, but is important to understand generally
what they do, and how changing the parameters will affect the functions' output.


In [ ]:
def resample_img_itk(itk_image, factors=[4.0, 4.0, 4.0]):
    """
    Resample with simple itk
    see https://gist.github.com/mrajchl/ccbd5ed12eb68e0c1afc5da116af614a#file-resample_itk_image-py-L6

    
    """

    # The pixel spacing of the input image
    original_spacing = itk_image.GetSpacing()
    
    # The size (number of pixels) of the original image
    original_size = itk_image.GetSize()

    # Compute the output size (number of pixels)
    # using the original image's size and spacing, and the desired
    # output pixel spacing
    out_size = [
        int(np.round(original_size[0] / factors[0])),
        int(np.round(original_size[1] / factors[1])),
        int(np.round(original_size[2] / factors[2]))]
    
    out_spacing = [
        original_spacing[0] * factors[0],
        original_spacing[0] * factors[1],
        original_spacing[0] * factors[2],
    ]

    # Set up the itk "filter" that will resample the image
    resample = sitk.ResampleImageFilter()
    
    # These two are the most important parameters for this use case
    # The desired pixel spacing of the output image
    resample.SetOutputSpacing(out_spacing)
    # The size (number of pixels) of the output image
    resample.SetSize(out_size)
    
    # The options below are less important (for this particular use-case)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform()) # This uses the identity transform
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    # Set the interpolation method.
    # Another option is: sitk.sitkNearestNeighbor
    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

def resample_arr_skimg(np_array, in_spacing=[1.0,1.0,1.0], factors=[4,4,4]):
    """
    Use downsample_local_mean from scikit-image to resample a numpy array.
    
    Since numpy array don't store anything about physical coordinates,
    in_spacing argument tells it what the physical spacing of the input is.
    
    Note this function only accepts integer values for the factors
    """
    return downscale_local_mean(np_array, tuple(factors), cval=0.0)

def resample_skimg(img, factors=[4,4,4]):
    """
    Use downsample_local_mean from scikit-image to resample a simple-itk image.
    
    Since numpy array don't store anything about physical coordinates,
    in_spacing argument tells it what the physical spacing of the input is.
    
    Note this function only accepts integer values for the factors
    """
    np_arr = sitk.GetArrayFromImage( img )
    in_spacing = img.GetSpacing()
    
    out_spacing = np.array(in_spacing) * np.array(factors)
    img_downsampled = downscale_local_mean(np_arr, tuple(factors), cval=0.0)
    itk_image = sitk.GetImageFromArray(img_downsampled.astype(np_arr.dtype))
    itk_image.SetOrigin((0, 0, 0))
    itk_image.SetSpacing(out_spacing)
    return itk_image

## Quiz

If we have an image (`img`) of size (in pixels) `(60,60,60)`, what will be the size
of `result1`?

``` 
result1 = resample_arr_skimg( img, factors=[3,3,3])
```

What will be the size of `result2`?

```
result2 = resample_arr_skimg( img, factors=[2,5,6])
```

Test your knowlege! Uncomment the cell below, run it, and check your result.

In [ ]:
# img = np.zeros([60,60,60])
# result1 = resample_arr_skimg( img, factors=[3,3,3])
# print( result1.shape )
# result2 = resample_arr_skimg( img, factors=[2,5,6])
# print( result2.shape )

In [ ]:
# Downsample jrc2018
factors = [4,4,4]
jrc18_path='../sampleImages/JRC2018_FEMALE_38um_iso_16bit.nrrd'
print('loading')
jrc18 = sitk.ReadImage(jrc18_path)
print( 'original spacing ', jrc18.GetSpacing())

print('downsampling')
# this spacing is equivalent to 4x downsampling
jrc18_down = resample_img_itk( jrc18, factors=factors)
print( 'new spacing ', jrc18_down.GetSpacing())
writer = sitk.ImageFileWriter()
writer.SetFileName('../sampleImages/jrc18_down.nrrd')
writer.Execute(jrc18_down)

In [ ]:
# Downsample jrc2018 using averaging

jrc18_down_avg = resample_skimg( jrc18, factors=factors)
writer = sitk.ImageFileWriter()
writer.SetFileName('../sampleImages/jrc18_down_avg.nrrd')
writer.Execute(jrc18_down_avg)

Now we have these three images in the `sampleImages` folder. 

* `JRC2018_FEMALE_38um_iso_16bit.nrrd`
* `jrc18_down.nrrd`
* `jrc18_down_avg.nrrd`

Open them all in Fiji. It will be helpful to set the brightness and contrast for all images to reasonable values with:

1. Set the brightness in contrast with `Image > Adjust > Brightness/Contrast` or `Ctrl + Shift + C`.
2. Press `Auto`
3. Press `Set`
4. Check the box next to `Propagate to all other open images`
5. Press `Ok`

Next run BigDataViewer with `Plugins > BigDataViewer > Open Current Image`, check all three boxes, and press "Ok".

By default, the images are overlayed.  Press `F` to turn off fused mode.  Now we're looking at one image at a time.  Cycle through the images by pressing `1`, `2`, and `3`.  The name of the image will be displayed at the top of the window.  Zoom in with the up-arrow.

* First compare `JRC2018_FEMALE_38um_iso_16bit.nrrd` and `jrc18_down_avg.nrrd`
* Next compare `JRC2018_FEMALE_38um_iso_16bit.nrrd` and `jrc18_down.nrrd`

What do you notice?

Finally, we'll also downsample another image because we will use the result later.

In [ ]:
# Downsample jrc2010
jrc10_path='../sampleImages/JFRCtemplate2010.nrrd'
jrc10 = sitk.ReadImage(jrc10_path)

jrc10_down = resample_skimg( jrc10, factors=factors)
writer = sitk.ImageFileWriter()
writer.SetFileName('../sampleImages/jrc10_down.nrrd')
writer.Execute(jrc10_down)

## Bonus : upsample

`resample_img_itk` can up-sample images too (factors can be less than one).

Change the value of `up_factors` below so that the image size is `(300, 150, 80)`.  What is the resulting spacing?

In [ ]:
up_factors = [1,1,1] # EDIT ME
jrc18_mid = resample_img_itk( jrc18_down, factors=up_factors)
print( jrc18_mid.GetSize() )
print( jrc18_mid.GetSpacing() )

## Bonus (optional, advanced?):

(I don't know how hard this is)

Explain what causes the difference between `jrc18_down.nrrd` and `jrc18_down_avg.nrrd`.

It is possible to account for this difference with metadata. How?